# Defining MDM

In [1]:
def mdm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="RMSE"):
    def error_check():
        rt = 0
        msg = ""
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)

        if (crit != "RMSE" and crit != "MAE" and crit != "MAPE"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    

    error_code = error_check()

    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return

    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    T = float(len(actual_lst))
    
    if crit == "RMSE":
        for actual, p1, p2 in zip(actual_lst, pred1_lst, pred2_lst):
            e1_lst.append((actual - p1) ** 2)
            e2_lst.append((actual - p2) ** 2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)

    if (crit == "MAE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)

    if (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
   
    mean_d = pd.Series(d_lst).mean()
    
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    MDM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    MDM_stat = harvey_adj*MDM_stat
    p_value = 2*t.cdf(-abs(MDM_stat), df = T - 1)
    
    dm_return = collections.namedtuple('mdm_return', 'MDM p_value')
    
    rt = dm_return(MDM = MDM_stat, p_value = p_value)
    
    return rt

# Importing real values

In [2]:
import random
import numpy as np
import pandas as pd
from pandas import read_csv
actual_lst = read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,22])
actual_lst=actual_lst.values
actual_lst=actual_lst.flatten()
actual_lst

array([3628.50988 , 3623.110046, 3619.090112, 3585.22007 , 3581.230076,
       3589.810193, 3642.310188, 3635.500075, 3644.310028, 3634.180057,
       3678.449974, 3670.960038, 3682.730125, 3699.19997 , 3697.410152,
       3708.450223, 3712.390023, 3678.490041, 3665.909803, 3697.610048,
       3695.29011 , 3711.270341, 3725.120403, 3726.700198, 3702.899807,
       3698.25993 , 3711.23996 , 3703.820033, 3740.510193, 3756.120132,
       3744.630076, 3760.199948, 3769.990008, 3737.83009 , 3783.040046,
       3811.550321, 3826.69002 , 3817.860253, 3810.780237, 3820.959962,
       3823.599998, 3788.730036, 3804.530185, 3859.749911, 3861.449907,
       3852.31017 , 3859.229917, 3870.900056, 3836.830119, 3830.499932,
       3778.050131, 3784.319996, 3843.090297, 3847.510024, 3872.41997 ,
       3894.559992, 3915.770101, 3918.350256, 3931.500242, 3925.989893,
       3937.2303  , 3950.43004 , 3933.610157, 3921.980085, 3930.41006 ,
       3902.919959, 3895.979958, 3928.650183, 3925.019913, 3861.

# Import the predictions of the proposed model

In [3]:
pred1_lst =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,23])
pred1_lst =pred1_lst.values
pred1_lst=pred1_lst.flatten()
pred1_lst

array([3602.0632, 3615.4163, 3600.1572, 3578.5388, 3573.979 , 3587.9202,
       3615.7075, 3634.257 , 3625.9097, 3628.5383, 3651.2463, 3669.2883,
       3674.3042, 3680.1736, 3684.9783, 3698.4763, 3702.9534, 3674.0935,
       3661.5718, 3682.657 , 3693.1719, 3697.1902, 3712.461 , 3715.1882,
       3700.0652, 3694.6643, 3699.2236, 3706.4263, 3724.8494, 3740.3547,
       3741.428 , 3747.814 , 3752.0544, 3741.3054, 3757.4067, 3796.7615,
       3812.4673, 3800.3127, 3796.3   , 3811.9465, 3810.3037, 3789.4153,
       3796.2205, 3831.703 , 3850.1685, 3839.7437, 3837.8833, 3846.5151,
       3841.483 , 3810.2122, 3779.6936, 3788.3955, 3824.5957, 3844.185 ,
       3853.7358, 3875.4915, 3894.477 , 3904.3298, 3913.2715, 3914.3772,
       3919.2727, 3927.5303, 3919.1716, 3909.5032, 3910.2493, 3890.903 ,
       3884.3086, 3907.505 , 3901.87  , 3869.605 , 3873.7227, 3892.6436,
       3868.4   , 3839.14  , 3845.1426, 3861.5088, 3885.1382, 3907.9246,
       3921.918 , 3932.4229, 3944.3994, 3957.943 , 

# BPNN

In [4]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,1])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-23.26267192749259, p_value=1.1589523013433525e-85)
crit：MAE， mdm_return(MDM=-32.861777609092734, p_value=6.410539556325623e-136)
crit：MAPE， mdm_return(MDM=-33.37014924275424, p_value=1.6988781859005983e-138)


# SVR

In [5]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,2])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-21.265620457135686, p_value=4.202169617523614e-75)
crit：MAE， mdm_return(MDM=-29.356662359611462, p_value=7.378468494428375e-118)
crit：MAPE， mdm_return(MDM=-30.12201180151065, p_value=7.656285789981458e-122)


# GRU

In [6]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,3])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-19.967526922226437, p_value=2.645056771937555e-68)
crit：MAE， mdm_return(MDM=-27.317934906414433, p_value=3.699733342998055e-107)
crit：MAPE， mdm_return(MDM=-27.277327927075948, p_value=6.058813938688991e-107)


# LSTM

In [7]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,4])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-18.663282630152445, p_value=1.4709477547214355e-61)
crit：MAE， mdm_return(MDM=-26.59758492439697, p_value=2.3622328274218927e-103)
crit：MAPE， mdm_return(MDM=-26.60102984856871, p_value=2.265174855286991e-103)


# Transformer

In [8]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,5])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.58226976185001, p_value=4.6250594998849836e-51)
crit：MAE， mdm_return(MDM=-21.393299484526242, p_value=8.938788413121413e-76)
crit：MAPE， mdm_return(MDM=-21.479345731834837, p_value=3.14760208497763e-76)


# Informer

In [9]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,6])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-12.586356787767498, p_value=2.2162810060488975e-32)
crit：MAE， mdm_return(MDM=-15.748195034977947, p_value=5.634026793746806e-47)
crit：MAPE， mdm_return(MDM=-15.831105359549012, p_value=2.2303248509446026e-47)


# TCN

In [10]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,7])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-14.18458371579463, p_value=1.4686489221330284e-39)
crit：MAE， mdm_return(MDM=-18.346183026950612, p_value=6.176563847323691e-60)
crit：MAPE， mdm_return(MDM=-18.408856931840912, p_value=2.954766819193393e-60)


# BiLSTM

In [11]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,8])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-22.938595597900335, p_value=6.062187985791052e-84)
crit：MAE， mdm_return(MDM=-31.93429045467514, p_value=3.448377192990257e-131)
crit：MAPE， mdm_return(MDM=-32.08312410058671, p_value=5.969839659204888e-132)


# CNN-BiLSTM-SAM

In [12]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,9])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-13.970131133888497, p_value=1.431358165210171e-38)
crit：MAE， mdm_return(MDM=-21.920374786674113, p_value=1.4834095404919516e-78)
crit：MAPE， mdm_return(MDM=-21.521758240090172, p_value=1.8813276979018866e-76)


# ICEEMDAN-SVR

In [13]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,10])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-24.77683986664436, p_value=1.0609111174089494e-93)
crit：MAE， mdm_return(MDM=-33.247639707255566, p_value=7.080028673799983e-138)
crit：MAPE， mdm_return(MDM=-33.334271315583706, p_value=2.580034792014708e-138)


# CEEMDAN-LSTM

In [14]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,11])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-12.759896958992055, p_value=3.8781667967135e-33)
crit：MAE， mdm_return(MDM=-16.036296021824782, p_value=2.2323223640827228e-48)
crit：MAPE， mdm_return(MDM=-15.880540410988882, p_value=1.2823466249117817e-47)


# ICEEMDAN-BiGRU

In [15]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,12])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-18.492764626861938, p_value=1.0998902314734876e-60)
crit：MAE， mdm_return(MDM=-23.903359837362647, p_value=4.610012297543188e-89)
crit：MAPE， mdm_return(MDM=-24.445275028876306, p_value=6.112178616763102e-92)


# VMD-BiLSTM

In [16]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,13])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-12.647096584609793, p_value=1.205972080678996e-32)
crit：MAE， mdm_return(MDM=-13.959014793627379, p_value=1.6099129898997596e-38)
crit：MAPE， mdm_return(MDM=-13.978670891867504, p_value=1.3077177296336887e-38)


# BiLSTM-SAM-TCN

In [17]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,14])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-14.381847712898779, p_value=1.7819867928815522e-40)
crit：MAE， mdm_return(MDM=-20.38228114843857, p_value=1.8108335579065793e-70)
crit：MAPE， mdm_return(MDM=-20.140566837774344, p_value=3.3140219471632894e-69)


# ICEEMDAN-BiLSTM-SAM-TCN

In [18]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,15])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.120146771749404, p_value=3.111754711331662e-26)
crit：MAE， mdm_return(MDM=-11.43781447783347, p_value=1.5877988374656916e-27)
crit：MAPE， mdm_return(MDM=-11.109193352310406, p_value=3.444674658403817e-26)


# ICEEMDAN-VMD(PSO)-SAM-TCN

In [19]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,16])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-13.35471352421533, p_value=8.931314654815756e-36)
crit：MAE， mdm_return(MDM=-16.83407996840858, p_value=2.608343150246779e-52)
crit：MAPE， mdm_return(MDM=-16.66580451358406, p_value=1.7848530401887995e-51)


# ICEEMDAN-VMD(PSO)-BiLSTM-SAM

In [20]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,17])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-8.478100197235321, p_value=1.8028551007043661e-16)
crit：MAE， mdm_return(MDM=-10.135246185634784, p_value=2.2221863660727463e-22)
crit：MAPE， mdm_return(MDM=-10.289918558493596, p_value=5.72087805121319e-23)


# ICEEMDAN-VMD(PSO)-BiLSTM-TCN

In [21]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,18])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-17.967341641143495, p_value=5.246809718707468e-58)
crit：MAE， mdm_return(MDM=-25.236486170879548, p_value=3.8534481239761625e-96)
crit：MAPE， mdm_return(MDM=-25.56335755446982, p_value=7.10771794084397e-98)


# CEEMDAN-VMD(PSO)-BiLSTM-SAM-TCN

In [22]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,19])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-9.583405751427206, p_value=2.503720037051461e-20)
crit：MAE， mdm_return(MDM=-8.05500580105942, p_value=4.344453182566468e-15)
crit：MAPE， mdm_return(MDM=-8.222761080198763, p_value=1.2487811487263741e-15)


# ICEEMDAN-VMD(PSO)-BiLSTM-SAM-CNN

In [23]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,20])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-10.682894865573036, p_value=1.710612876878668e-24)
crit：MAE， mdm_return(MDM=-12.19627118913245, p_value=1.0604032988790666e-30)
crit：MAPE， mdm_return(MDM=-11.51571725733641, p_value=7.593018266093913e-28)


# ICEEMDAN-VMD(PSO)-LSTM-SAM-TCN

In [24]:
pred2_lst  =read_csv('./True value of S&P 500 High and all model predictions.csv', index_col=0,usecols=[0,21])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-19.35925680389981, p_value=3.810047320920196e-65)
crit：MAE， mdm_return(MDM=-20.29574219415889, p_value=5.130765233439862e-70)
crit：MAPE， mdm_return(MDM=-19.654448648631767, p_value=1.1237285112876478e-66)
